In [2]:
import $ivy.`org.apache.spark:spark-core_2.11:2.4.4`
import $ivy.`org.apache.spark:spark-sql_2.11:2.4.4`
import $ivy.`org.apache.spark:spark-streaming_2.11:2.4.4`
import $ivy.`com.microsoft.azure:azure-eventhubs-spark_2.11:2.3.13`
import $ivy.`io.delta:delta-core_2.11:0.3.0`
import $ivy.`log4j:log4j:1.2.17`

import $ivy.$                                       

import $ivy.$                                      

import $ivy.$                                            

import $ivy.$                                                      

import $ivy.$                               

import $ivy.$                   

## Importando Dependências

In [11]:
// Spark
import org.apache.spark.SparkConf
import org.apache.spark.sql.SparkSession

// Event Hubs
import org.apache.spark.eventhubs.{EventHubsConf,EventPosition}

// Delta Lake
import io.delta.tables.DeltaTable

// Log4j
import org.apache.log4j.{Level,Logger}
Logger.getLogger("org").setLevel(Level.OFF)
Logger.getLogger("akka").setLevel(Level.OFF)
Logger.getRootLogger.setLevel(Level.OFF)

import org.apache.spark.SparkConf

import org.apache.spark.sql.SparkSession

// Event Hubs

import org.apache.spark.eventhubs.{EventHubsConf,EventPosition}

// Delta Lake

import io.delta.tables.DeltaTable

// Log4j

import org.apache.log4j.{Level,Logger}


In [4]:
// Criando Contexto de Spark
val sparkConfig = new SparkConf().setAll(Seq(("spark.driver.memory","2g")))
val spark = SparkSession.builder.config(sparkConfig).master("local[*]").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


sparkConfig: SparkConf = org.apache.spark.SparkConf@49ee418b
spark: SparkSession = org.apache.spark.sql.SparkSession@65788322

In [17]:
import org.apache.spark.sql.functions._
import spark.implicits._
import org.apache.spark.sql.types._
import org.apache.spark.sql.expressions._

import org.apache.spark.sql.functions._

import spark.implicits._

import org.apache.spark.sql.types._

import org.apache.spark.sql.expressions._

In [5]:
// Criando Conexão com o Event Hubs
val key =  "Endpoint=sb://stockhub.servicebus.windows.net/;SharedAccessKeyName=listen;SharedAccessKey=q/ZEO4MaguioHiH2vQof02xdsGZVGE1V5RrE9IldOWU=;EntityPath=ev-stock"
val evConf = EventHubsConf(key).setStartingPosition(EventPosition.fromStartOfStream)

key: String = "Endpoint=sb://stockhub.servicebus.windows.net/;SharedAccessKeyName=listen;SharedAccessKey=q/ZEO4MaguioHiH2vQof02xdsGZVGE1V5RrE9IldOWU=;EntityPath=ev-stock"
evConf: EventHubsConf = org.apache.spark.eventhubs.EventHubsConf@1d9d998c

In [13]:
val rawDF = spark.read.format("eventhubs").options(evConf.toMap).load()

rawDF: org.apache.spark.sql.package.DataFrame = [body: binary, partition: string ... 7 more fields]

In [31]:
rawDF
    .select($"body".cast(StringType))
    .select(get_json_object($"body","$.data") as "data")
    .select(parseMultipleJsons($"data") as "data")
    .select(explode($"data") as "data")
    .select(json_tuple($"data",columns:_*))
    .toDF(columns:_*)
    .printSchema

root
 |-- symbol: string (nullable = true)
 |-- name: string (nullable = true)
 |-- currency: string (nullable = true)
 |-- price: string (nullable = true)
 |-- price_open: string (nullable = true)
 |-- day_high: string (nullable = true)
 |-- day_low: string (nullable = true)
 |-- 52_week_low: string (nullable = true)
 |-- day_change: string (nullable = true)
 |-- change_pct: string (nullable = true)
 |-- close_yesterday: string (nullable = true)
 |-- market_cap: string (nullable = true)
 |-- volume: string (nullable = true)
 |-- volume_avg: string (nullable = true)
 |-- shares: string (nullable = true)
 |-- timezone_name: string (nullable = true)
 |-- gmt_offset: string (nullable = true)
 |-- last_trade_time: string (nullable = true)



In [21]:
// Para cada linha da nossa tabela, deixa em um formato mais fácil de manipular
val parseMultipleJsons = udf((json : String) => {
    val jsonSet = "[]".toSet
    
    json
        .filterNot(jsonSet)
        .replaceAll("},","}-,},")
        .split("},")
        .map(r => r.replaceAll("}-,","}").trim)
        .filter(r => r.length > 0)
})

parseMultipleJsons: UserDefinedFunction = UserDefinedFunction(
  <function1>,
  ArrayType(StringType, true),
  Some(List(StringType))
)

In [32]:
def parseStreamDF(df : org.apache.spark.sql.DataFrame) : org.apache.spark.sql.DataFrame = {
    df
        .select($"body".cast(StringType))
        .select(get_json_object($"body","$.data") as "data")
        .select(parseMultipleJsons($"data") as "data")
        .select(explode($"data") as "data")
        .select(json_tuple($"data",columns:_*))
        .toDF(columns:_*)
        .withColumn("price",$"price".cast(DoubleType))
        .withColumn("day_high",$"day_high".cast(DoubleType))
        .withColumn("day_low",$"day_high".cast(DoubleType))
        .withColumn("52_week_low",$"52_week_low".cast(DoubleType))
        .withColumn("day_change",$"day_change".cast(DoubleType))
        .withColumn("change_pct",$"change_pct".cast(DoubleType))
        .withColumn("close_yesterday",$"close_yesterday".cast(DoubleType))
        .withColumn("market_cap",$"market_cap".cast(LongType))
        .withColumn("volume",$"volume".cast(LongType))
        .withColumn("volume_avg",$"volume_avg".cast(DoubleType))
        .withColumn("shares",$"shares".cast(LongType))
        .withColumn("gmt_offset",$"shares".cast(IntegerType))
        .withColumn("last_trade_time",to_timestamp($"last_trade_time"))
        .withColumn("brazil_time_zone",from_utc_timestamp(to_utc_timestamp($"last_trade_time","America/New_York"),"America/Sao_Paulo"))
}

defined function parseStreamDF

In [33]:
val columns = Array("symbol","name","currency","price","price_open","day_high","day_low","52_week_low","day_change","change_pct",
                 "close_yesterday","market_cap","volume","volume_avg","shares","timezone_name","gmt_offset","last_trade_time")
val stockDF2 = parseStreamDF(rawDF)

columns: Array[String] = Array(
  "symbol",
  "name",
  "currency",
  "price",
  "price_open",
  "day_high",
  "day_low",
  "52_week_low",
  "day_change",
  "change_pct",
  "close_yesterday",
  "market_cap",
  "volume",
  "volume_avg",
  "shares",
  "timezone_name",
  "gmt_offset",
  "last_trade_time"
)
stockDF2: org.apache.spark.sql.package.DataFrame = [symbol: string, name: string ... 17 more fields]

In [34]:
val stockDataPath = "/home/vitor/Documents/Big-Data/StockStreaming/Analyzer/stockdataV3"

stockDF2
    .withColumn("row",row_number().over(Window.partitionBy($"symbol").orderBy($"last_trade_time".desc)))
    .where("row = 1")
    .drop("row")
    .write
    .format("delta")
    .option("path",stockDataPath)
//     .save("/home/vitor/Documents/Big-Data/StockStreaming/Analyzer/stockdataV2")
    .saveAsTable("stockV3")

stockDataPath: String = "/home/vitor/Documents/Big-Data/StockStreaming/Analyzer/stockdataV3"

In [41]:
spark.read.table("stockV3").showHTML()

symbol,name,currency,price,price_open,day_high,day_low,52_week_low,day_change,change_pct,close_yesterday,market_cap,volume,volume_avg,shares,timezone_name,gmt_offset,last_trade_time,brazil_time_zone
NVDA,NVIDIA Corporation,USD,172.53,176.10,177.29,177.29,124.46,-2.31,-1.32,174.84,105070772224,7442704,6493600.0,609000000,America/New_York,609000000,2019-09-24 16:00:...,2019-09-24 17:00:...
AMZN,"Amazon.com, Inc.",USD,1741.61,1790.60,1795.7,1795.7,1307.0,-43.69,-2.45,1785.3,861497851904,4315284,3076533.0,494656000,America/New_York,494656000,2019-09-24 16:00:...,2019-09-24 17:00:...
GOOG,Alphabet Inc.,BRL,1218.76,1240.00,1246.56,1246.56,970.11,-15.27,-1.24,1234.03,845311180800,1372965,1245316.0,347344992,America/New_York,347344992,2019-09-24 16:00:...,2019-09-24 17:00:...
FB,"Facebook, Inc.",USD,181.28,187.98,188.04,188.04,123.02,-5.54,-2.97,186.82,517182783488,15753694,1.1908716E7,2405720064,America/New_York,-1889247232,2019-09-24 16:00:...,2019-09-24 17:00:...


In [49]:
// Recuperando tabela Delta
DeltaTable
    .forPath(stockDataPath)
    .history
    .showHTML()

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend
2,2019-09-24 18:48:...,null,null,DELETE,Map(predicate -> ...,null,null,null,1,null,false
1,2019-09-24 18:40:...,null,null,UPDATE,Map(predicate -> ...,null,null,null,0,null,false
0,2019-09-24 18:36:...,null,null,WRITE,Map(mode -> Overw...,null,null,null,null,null,false


In [43]:
spark.read.format("delta").option("versionAsOf","0").load(stockDataPath).showHTML()

symbol,name,currency,price,price_open,day_high,day_low,52_week_low,day_change,change_pct,close_yesterday,market_cap,volume,volume_avg,shares,timezone_name,gmt_offset,last_trade_time,brazil_time_zone
NVDA,NVIDIA Corporation,USD,172.53,176.10,177.29,177.29,124.46,-2.31,-1.32,174.84,105070772224,7442704,6493600.0,609000000,America/New_York,609000000,2019-09-24 16:00:...,2019-09-24 17:00:...
AMZN,"Amazon.com, Inc.",USD,1741.61,1790.60,1795.7,1795.7,1307.0,-43.69,-2.45,1785.3,861497851904,4315284,3076533.0,494656000,America/New_York,494656000,2019-09-24 16:00:...,2019-09-24 17:00:...
GOOG,Alphabet Inc.,USD,1218.76,1240.00,1246.56,1246.56,970.11,-15.27,-1.24,1234.03,845311180800,1372965,1245316.0,347344992,America/New_York,347344992,2019-09-24 16:00:...,2019-09-24 17:00:...
FB,"Facebook, Inc.",USD,181.28,187.98,188.04,188.04,123.02,-5.54,-2.97,186.82,517182783488,15753694,1.1908716E7,2405720064,America/New_York,-1889247232,2019-09-24 16:00:...,2019-09-24 17:00:...


In [40]:
// Realizando Update na tabela delta
val deltaStock = DeltaTable
                    .forPath(stockDataPath)

deltaStock.updateExpr(
    "symbol = 'GOOG'", Map("currency" -> "'BRL'")
)

deltaStock: DeltaTable = io.delta.tables.DeltaTable@78c08041

In [47]:
// Deletando um registro na tabela delta
deltaStock.delete("symbol = 'NVDA'")

In [48]:
deltaStock.toDF.showHTML()

symbol,name,currency,price,price_open,day_high,day_low,52_week_low,day_change,change_pct,close_yesterday,market_cap,volume,volume_avg,shares,timezone_name,gmt_offset,last_trade_time,brazil_time_zone
AMZN,"Amazon.com, Inc.",USD,1741.61,1790.60,1795.7,1795.7,1307.0,-43.69,-2.45,1785.3,861497851904,4315284,3076533.0,494656000,America/New_York,494656000,2019-09-24 16:00:...,2019-09-24 17:00:...
GOOG,Alphabet Inc.,BRL,1218.76,1240.00,1246.56,1246.56,970.11,-15.27,-1.24,1234.03,845311180800,1372965,1245316.0,347344992,America/New_York,347344992,2019-09-24 16:00:...,2019-09-24 17:00:...
FB,"Facebook, Inc.",USD,181.28,187.98,188.04,188.04,123.02,-5.54,-2.97,186.82,517182783488,15753694,1.1908716E7,2405720064,America/New_York,-1889247232,2019-09-24 16:00:...,2019-09-24 17:00:...


In [63]:
deltaStock.history.showHTML()

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend
3,2019-09-22 18:28:...,null,null,WRITE,Map(mode -> Overw...,null,null,null,2,null,false
2,2019-09-22 18:23:...,null,null,DELETE,Map(predicate -> ...,null,null,null,1,null,false
1,2019-09-22 18:22:...,null,null,UPDATE,Map(predicate -> ...,null,null,null,0,null,false
0,2019-09-22 18:18:...,null,null,WRITE,Map(mode -> Overw...,null,null,null,null,null,false


## Lendo Dados do Event Hub em Real Time

In [51]:
val checkpoint = "/home/vitor/Documents/Big-Data/StockStreaming/Analyzer/stockCheckpointV2"
val rawStreamDF = spark.readStream.format("eventhubs").options(evConf.toMap).load()
val parsedStreamDF = parseStreamDF(rawStreamDF)

checkpoint: String = "/home/vitor/Documents/Big-Data/StockStreaming/Analyzer/stockCheckpointV2"
rawStreamDF: org.apache.spark.sql.package.DataFrame = [body: binary, partition: string ... 7 more fields]
parsedStreamDF: org.apache.spark.sql.package.DataFrame = [symbol: string, name: string ... 17 more fields]

In [52]:
parsedStreamDF
    .writeStream
    .option("checkpoinLocation",checkpoint)
    .foreachBatch((microBatchOutputDF : org.apache.spark.sql.DataFrame, batchId : Long) => {
        // Um batch pode contar várias ações duplicadas
        val outputStream = microBatchOutputDF
                                    .withColumn("row",row_number().over(Window.partitionBy($"symbol").orderBy($"last_trade_time".desc)))
                                    .where("row = 1")
                                    .drop("row")
        
        deltaStock
            .as("origem")
            .merge(
                outputStream.as("att"),
                "origem.symbol = att.symbol"
            )
            .whenMatched().updateAll()
            .whenNotMatched().insertAll()
            .execute()
        
    }).start().awaitTermination()

: 

## Printando Spark Dataframe como HTML

In [15]:
implicit class RichDF(val ds: org.apache.spark.sql.DataFrame) {
    def showHTML(limit:Int = 20, truncate: Int = 20) = {
        import xml.Utility.escape
        val data = ds.take(limit)
        val header = ds.schema.fieldNames.toSeq        
        val rows: Seq[Seq[String]] = data.map { row =>
          row.toSeq.map { cell =>
            val str = cell match {
              case null => "null"
              case binary: Array[Byte] => binary.map("%02X".format(_)).mkString("[", " ", "]")
              case array: Array[_] => array.mkString("[", ", ", "]")
              case seq: Seq[_] => seq.mkString("[", ", ", "]")
              case _ => cell.toString
            }
            if (truncate > 0 && str.length > truncate) {
              // do not show ellipses for strings shorter than 4 characters.
              if (truncate < 4) str.substring(0, truncate)
              else str.substring(0, truncate - 3) + "..."
            } else {
              str
            }
          }: Seq[String]
        }

        publish.html(s""" <table>
                <tr>
                 ${header.map(h => s"<th>${escape(h)}</th>").mkString}
                </tr>
                ${rows.map { row =>
                  s"<tr>${row.map{c => s"<td>${escape(c)}</td>" }.mkString}</tr>"
                }.mkString}
            </table>
        """)        
    }
}

defined class RichDF

In [62]:
deltaStock
    .toDF
    .limit(1)
    .withColumn("price",$"price".cast(StringType))
    .write
    .format("delta")
    .mode("append")
    .save(stockDataPath)

: 